In [3]:
## Some random imorts
import h5py
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from os.path import join
import cv2
import pandas as pd
import glob
from os.path import basename, dirname,join
import os
from tqdm import tqdm
import seaborn as sns

In [4]:
## import functions for bootstrap tests
from bootstrap_TCGA_improvement_test import CV_bootstrap_improvement_test
from bootstrap_TCGA_bias_test import CV_bootstrap_bias_test


## Example CSV files for the results

    Each pd.DataFrame contains the following columns:
        * sens_attr: sensitive attributes
        * prob: model probabilities
        * label: ground truth labels
        * pred:  model predictions
        * [A slide ID]

In [5]:
baseline_csvs    = [f'bootstrap_example_csvs/baseline/inference_results_fold{i}.csv' for i in range(4)]
corrected_csvs   = [f'bootstrap_example_csvs/corrected/inference_results_fold{i}.csv' for i in range(4)]

dfs_baseline = [pd.read_csv(csv) for csv in baseline_csvs]
dfs_corrected = [pd.read_csv(csv) for csv in corrected_csvs]


In [6]:
dfs_baseline[0].head()

,Unnamed: 0,label,logits,prob,file,sens_attr,subjIDs
0,0,0.0,0.855903,0.701804,/n/data2/hms/dbmi/kyu/lab/datasets/MayoBrain/G...,M,G-002
1,1,1.0,-1.957226,0.123768,/n/data2/hms/dbmi/kyu/lab/datasets/MayoBrain/G...,F,G-005
2,2,1.0,-3.703713,0.024040,/n/data2/hms/dbmi/kyu/lab/datasets/MayoBrain/G...,F,G-014
3,3,0.0,-0.621319,0.349482,/n/data2/hms/dbmi/kyu/lab/datasets/MayoBrain/G...,F,G-019
4,4,1.0,2.343101,0.912384,/n/data2/hms/dbmi/kyu/lab/datasets/MayoBrain/G...,F,G-021


Add binary predictions

In [7]:
##################
for df in dfs_baseline:
    df['pred'] = df['prob'].apply(lambda x: 1 if x > 0.5 else 0)
for df in dfs_corrected:
    df['pred'] = df['prob'].apply(lambda x: 1 if x > 0.5 else 0)


Remove rows without sensitive attribute

In [8]:
dfs_baseline = [df.loc[~df['sens_attr'].isna()] for df in dfs_baseline]
dfs_corrected = [df.loc[~df['sens_attr'].isna()] for df in dfs_corrected]

### Test for significant bias
        df_p_worse:     If the underpriviledged group has lower performance than the priviledged group (for each  performance metric)
        df_p_better:    If the underpriviledged group has HIGHER performance than the priviledged group (for each  performance metric)
        fairResult:     the original performance & fairnes metrics
        df_CI:          The confidence interval of the performance & fairnes metrics

In [9]:
aggregate_method = 'concatenate' # can also be 'concatenate'
n_bootstrap=1000 ## should be more than 1000 for valid analysis
privileged_group = None # if given, fix the priviledged group. If None, the priviledged group will be the group with the best performance (may be different for each metric)
df_p_worse_baseline, df_p_better_baseline, fairResult_baseline, df_CI_baseline =  CV_bootstrap_bias_test(
    dfs_baseline, privileged_group=privileged_group, n_bootstrap=n_bootstrap,aggregate_method=aggregate_method)
df_p_worse_corrected, df_p_better_corrected, fairResult_corrected, df_CI_corrected =  CV_bootstrap_bias_test(
    dfs_corrected, privileged_group=privileged_group, n_bootstrap=n_bootstrap,aggregate_method=aggregate_method)
# def CV_bootstrap_improvement_test(
#     dfs_baseline, dfs_corrected, privileged_group=None, n_bootstrap=1000,aggregate_method='fisher',add_perf_difference=args.add_perf_difference)

100%|██████████| 1000/1000 [00:17<00:00, 56.08it/s]


In [10]:
## If you use fisher, there will be p-values for each fold, plus a combined p-value
df_p_worse_baseline

,PQD,PQD(class),EPPV,ENPV,DPM(Positive),DPM(Negative),EOM(Positive),EOM(Negative),AUCRatio,EOpp0,...,PPV_diff,NPV_diff,PR_diff,NR_diff,BAcc_diff,FPR_diff,FNR_diff,OverAllAcc_diff,OverAllAUC_diff,TOTALACC_diff
pval,0.211,0.211,0.265,0.486,0.897,0.892,0.462,0.269,0.981,0.275,...,0.267,0.482,0.891,0.891,0.208,0.725,0.519,1.0,1.0,0.212


### Test for significant improvement
        df_p_better: pd.DataFrame, the p-values for significant improvement
        df_p_worse: pd.DataFrame, the p-values for significant worsening

In [11]:
aggregate_method = 'concatenate' # can also be 'concatenate'
n_bootstrap=1000 ## should be more than 1000 for valid analysis
privileged_group = None # if given, fix the priviledged group. If None, the priviledged group will be the group with the best performance (may be different for each metric)
ID_col = 'subjIDs'

df_improv, df_p_better, df_p_worse =  CV_bootstrap_improvement_test(
    dfs_baseline, dfs_corrected, privileged_group=privileged_group, 
    n_bootstrap=n_bootstrap,aggregate_method=aggregate_method,
    ID_col=ID_col)

100%|██████████| 1000/1000 [00:53<00:00, 18.77it/s]
/home/FAIR-Tuning/bootstrap_significant_test/bootstrap_TCGA_improvement_test.py:242: PerformanceWarning: indexing past lexsort depth may impact performance.
  bootstrap_values = df_improvement_bootstrap[col].loc[i].dropna()


In [12]:
df_p_better

,,AUC,ACC,TPR,TNR,PPV,NPV,PR,NR,BAcc,FPR,...,PPV_diff,NPV_diff,PR_diff,NR_diff,BAcc_diff,FPR_diff,FNR_diff,OverAllAcc_diff,OverAllAUC_diff,TOTALACC_diff
sensitiveAttr,group_type,,,,,,,,,,,,,,,,,,,,,
F,unspecified,0.328,0.167,0.068,0.648,0.313,0.060,0.169,0.895,0.199,0.648,...,0.452,0.085,0.948,0.941,0.229,0.346,0.957,1.0,1.0,0.204
M,unspecified,0.003,0.571,0.792,0.321,0.322,0.631,0.840,0.266,0.455,0.321,...,0.452,0.085,0.948,0.941,0.229,0.346,0.957,1.0,1.0,0.204
